<a href="https://colab.research.google.com/github/SUNNYTHAKURCODE/MACHINELEARNING/blob/master/LSTM_Text_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data="""Where are they flying\n
    Glowing like a sun that is dying\n
  Lightening the world by their burning\n
  Shocking the world by their adjourning\n
  Inspiring the generation by always trying\n
  Why Today these Wings of Fire have made us crying\n
  Once delivering bundle of newspaper lying\n
  Once learning the meaning of life overlying\n
  Once getting up from the disappointment of not flying\n
  Now with his own wings defying to stop flying\n
  Building the rockets that made satellites floating\n
  Guiding the projects that made other countries noting\n
  Imagining the future while dreaming, (as president)\n
  Fulfilling the dream by dedicated working\n
  Teaching us the way to dream freely\n
  Uniting us all to dream togetherly\n
  May be to teach one last lesson to keep dreaming\n
  So that the fire of the wings keeps burning\n
  That these wings are never going to stop flying\n
  To achieve the dream they are just going to make us vying\n
  These are the Wings Of Fire that are never dying"""


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np 

tokenizer = Tokenizer()

def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization	
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences 
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words

def create_model(predictors, label, max_sequence_len, total_words):
	
	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	# model.add(Dropout(0.2))
	model.add(LSTM(100))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
	model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
	print (model.summary())
	return model 

def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict_classes(token_list, verbose=0)
		
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text


predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
print (generate_text("we naughty", 3, max_sequence_len))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
140/140 [==============================] - 1s 6ms/step - loss: 4.5512 - accuracy: 0.0429
Epoch 2/100
140/140 [==============================] - 0s 1ms/step - loss: 4.5314 - accuracy: 0.1000
Epoch 3/100
 32/140 [=====>........................] - ETA: 0s - loss: 4.5081 - accuracy: 0.0938

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


140/140 [==============================] - 0s 1ms/step - loss: 4.4778 - accuracy: 0.1000
Epoch 4/100
140/140 [==============================] - 0s 1ms/step - loss: 4.2998 - accuracy: 0.1000
Epoch 5/100
140/140 [==============================] - 0s 1ms/step - loss: 4.1770 - accuracy: 0.1000
Epoch 6/100
140/140 [==============================] - 0s 1ms/step - loss: 4.1196 - accuracy: 0.1000
Epoch 7/100
140/140 [==============================] - 0s 1ms/step - loss: 4.1011 - accuracy: 0.1000
Epoch 8/100
140/140 [==============================] - 0s 1ms/step - loss: 4.0840 - accuracy: 0.1000
Epoch 9/100
140/140 [==============================] - 0s 1ms/step - loss: 4.0658 - accuracy: 0.1000
Epoch 10/100
140/140 [==============================] - 0s 1ms/step - loss: 4.0645 - accuracy: 0.1000
Epoch 11/100
140/140 [==============================] - 0s 1ms/step - loss: 4.0533 - accuracy: 0.1000
Epoch 12/100
140/140 [==============================] - 0s 1ms/step - loss: 4.0468 - accuracy: 0.1000